# Benefits and drawbacks of formulating the COVID-19 SEIR model as an SDE rather than an ODE

Robert Perrotta\
1 May 2020

Whereas an ordinary differential equation (ODE) defines a determinstic process described by an initial condition and a function to compute the derivative for all values in the space, a stochastic differential equation (SDE) adds a noise term so the evolution of the process is no longer deterministic. Typically, the expectation of the SDE is an ODE and as the noise term decreases, the difference between the two vanishes. Despite the similarity, their implementation in pymc3 is completely different. I believe a SDE formulation can more appropriately capture the dynamics in the data and will provide more tractible sampling for uncertainty characterization but early work suggests it is at least as sensitive to initial parameter guesses as the ODE forumlation. To that end, I propose using nevergrad to find a robust initial fit from which to start the pymc3 sampling.

## A vectorized SEIR model

To take advantage of fast arithmetic operations in python using numpy, we need to formulate our model in terms of fewer, larger arrays, rather than many small arrays or python basic data types. In fact, we can describe the entire state of our system in one array, which we will cal $y$. To do this, we can either keep a second dimension of indexed states (e.g. susceptible, exposed, hidden infectious with no symptoms, confirmed infectious with no symptoms, etc.) or we can build a many-dimensional array each to index separate pieces of state information (symptom severity, hidden or detected, alive or dead, etc.). Though the latter will involve allocating space for some impossible combinations of states (e.g. dead and recovered, susceptible and confirmed) it is significantly more extensible and maintainable and is therefore the method we chose.

We define the following dimensions for our state array:
- disease state: susceptible, exposed, infectious with no symptoms, light symptoms, severe symptoms (requiring hospitalization), critical symptoms (requiring intensive therapy)
- age: age groups (boundaries TBD with hopes of aligning to age grouped case and lethality statistics)
- detection status: hidden or confirmed
- recovery status: not recovered or recovered
- health status: alive or deceased
- time

The value in each array element is a continuous analog for the number of individuals in that state.

$$
[\rm{state}\times\rm{age}\times\rm{detected}\times\rm{recovered}\times\rm{dead}\times\rm{time}]
$$

In this way we can describe the ODE/SDE in terms of a single $dy$ array constructed from a few simple components.

All living persons have a chance of dying in a time step, $\mu$. As we do not model deaths from other sources, and we assume asymptomatic disease is non-lethal, $\mu(\rm{state}\in[\rm{susceptible, exposed, asymptomatic}])=0$ and $\mu(\rm{recovered})=0$. With this, fatality rate is simply $\mu y(!\rm{dead})$. We define $\mu$ to depend on age and detection status, but not on time. As such, $\mu$ has dimensionality $[(\rm{state}-3)\times\rm{age}\times\rm{detected}\times1\times1\times1]$.

In a similar way, recovery from the disease is modeled with a simgle parameter $\gamma$. Since individuals cannot recover who have not yet been exposed or who are deceased, $\gamma(\rm{state}=\rm{susceptible})=0$ and $\gamma(\rm{dead})=0$. As before, the recovery rate becomes $\gamma y(!\rm{recovered})$. We define $\gamma$ to depend on age and detection status, but not on time. As such, $\gamma$ has dimensionality $[(\rm{state}-1)\times\rm{age}\times\rm{detected}\times1\times1\times1]$.

Testing works similarly, following the parameter $\theta$. We assume anyone exposed has a chance of being tested and that recovered and deceased individuals are not tested. (**This is one of the more tenuous assumptions in the model.** Are cases being identified post-mortem? Are serological surveys contributing to the case counts?) Under these assumptions $\theta(\rm{state}=\rm{susceptible})=0$, $\theta(\rm{recovered})=0$, and $\theta(\rm{dead})=0$ and the detection rate is $\theta y(!\rm{detected})$. We define $\theta$ to depend on symptom level, but not on age or time. As such, $\theta$ has dimensionality $[(\rm{state}-1)\times1\times1\times1\times1\times1]$.

$$
[\rm{state}\times\rm{age}\times\rm{detected}\times\rm{recovered}\times\rm{dead}\times\rm{time}]
$$

Disease progression applies to all exposed and infectious states except the last (which has no subsequent state to which it can progress) and is described by the parameter $\sigma$. We freeze progression for recovered and deceased individuals by setting $\sigma(\rm{recovered})=\sigma(\rm{dead})=0$. Therefore progression rate is $\sigma y(\rm{state}[1:-1])$ (using python's slice notation where $[1:-1]$ drops the first and last elements). We define $\sigma$ to depend on symptom level, but not on age or time. As such, $\sigma$ has dimensionality $[(\rm{state}-2)\times\rm{age}\times\rm{detected}\times1\times1\times1]$.

New exposures are governed by the parameter $\beta$. Infectious individuals have a probability of encountering another individual and passing the virus that depends on their symptom level, detection status, and age. Additionally, it varies over time as social distancing and lockdown measures are enacted. Here, we model it as constant over each of three "eras." We divide those at March 8th, when Italy issued a National lockdown, and March 21st, when Italy closed all non-essential businesses. We expect beta to be reduced at each transition but place no such restriction on the model. New exposure rate is therefore $\frac{y(\rm{state}=\rm{susceptible})}{N} \Sigma{\beta y(\rm{state}!=\rm{susceptible}, !\rm{recovered}, !\rm{dead})}$ where the sum is over all dimensions but time. This assumes equal mixing and contagiousness between all groups though one could conceivably implement this such that $\beta$ defined a kind of matrix of transmission which could capture hypotheses such as infectious individuals are most likely to pass the virus to people of a similar age. We define $\beta$ to depend on symptom level and detection status, but not on age. As such, $\beta$ has dimensionality $[(\rm{state}-2)\times1\times\rm{detected}\times1\times1\times\rm{eras}]$.

And lastly, we define the initial conditions of this model in terms of the number of exposed individuals, $e_0$, where the number susceptible is therefore $N(\rm{age})-e_0$ and all other categories are zero. Therefore $e_0$ has dimensionality $[1\times\rm{age}\times1\times1\times1\times1]$.

All told, the number of parameters in the model is therefore $2(\rm{state}-3)\times\rm{age} + 2(\rm{state}-1)\times\rm{age} + (\rm{state}-1) +  + 2(\rm{state}-2)\times\rm{eras} + \rm{age} = 4\rm{state}\times\rm{age} - 7\rm{age} + 2\rm{state}\times{eras} - 4\rm{eras} + \rm{state} - 1$. For our current implementation with 6 states, 2 age groups, and 3 eras, we have 63 parameters.

Notes:
- possible SDE formulation using Multinomial for more accurate transition randomness
- April 16, 26 stats
- Mobility datasets as a proxy for beta reduction
- Fraction of cases doesn't match up. Is Lombardy experiencing more critical cases? Is this a symptom of a bug in the model? Are these stats from known cases or estimates of all cases from sero-surveys?